# 1. 데이터 읽어오기

In [1]:
# import pandas as pd

# claim_data_path = "./data/patent/kipris_claim_patent.csv"
# claim_df = pd.read_csv(claim_data_path)
# claim_df = claim_df.drop(columns=["Unnamed: 0"])
# claim_df.head()

# 2. mecab-ko-dic

In [64]:
# path = "/home/ui/project/graduate_paper/korpatElectra/KIPI-KorPatELECTRA/pat_all_mecab_dic.csv"
# user_dict = pd.read_csv(path)
# len(user_dict), user_dict.head()

(6663692,
     실시예  Unnamed: 1  Unnamed: 2  Unnamed: 3  NNG  *  F 실시예.1 *.1 *.2 *.3 *.4
 0    일측         NaN         NaN         NaN  NNG  *  T    일측   *   *   *   *
 1   청구항         NaN         NaN         NaN  NNG  *  T   청구항   *   *   *   *
 2   제어부         NaN         NaN         NaN  NNG  *  F   제어부   *   *   *   *
 3  어드레스         NaN         NaN         NaN  NNG  *  F  어드레스   *   *   *   *
 4  제조방법         NaN         NaN         NaN  NNG  *  T  제조방법   *   *   *   *)

In [3]:
from konlpy.tag import Mecab

sentence_org = "본 고안은 주로 일회용 합성세제액을 집어넣어 밀봉하는 세제액포의 내부를 원호상으로 열중착하되 세제액이 배출되는 절단부 쪽으로 내벽을 협소하게 형성하여서 내부에 들어있는 세제액을 잘짜질 수 있도록 하는 합성세제 액포에 관한 것이다."
m = Mecab()  # dicpath="/usr/local/lib/mecab/dic/mecab-ko-dic"
sentence_kipi = " ".join(m.morphs(sentence_org))
sentence_kipi

'본 고안 은 주로 일회용 합성세제액 을 집어넣 어 밀봉 하 는 세제액포 의 내부 를 원호 상 으로 열중 착하 되 세제액 이 배출 되 는 절단부 쪽 으로 내벽 을 협소 하 게 형성 하 여서 내부 에 들 어 있 는 세제액 을 잘 짜 질 수 있 도록 하 는 합성세제 액포 에 관한 것 이 다 .'

# 3. 토크나이저

In [11]:
import importlib.util

# 정확한 tokenizer.py 경로 지정
tokenizer_path = "./KIPI-KorPatELECTRA/tokenizer.py"

# 모듈 불러오기
spec = importlib.util.spec_from_file_location("custom_tokenizer", tokenizer_path)
tokenization = importlib.util.module_from_spec(spec)
spec.loader.exec_module(tokenization)

# FullTokenizer 테스트
tokenizer = tokenization.FullTokenizer(vocab_file="./KIPI-KorPatELECTRA/vocab.txt", do_lower_case=False)
tokenizer_vocab = tokenizer.vocab

In [12]:
# 사용자 사전이 추가된 mecab-ko 형태소 분리기로 1차 분리
m = Mecab()
sentence_kipi = " ".join(m.morphs(sentence_org))

# 2차 분리
tokens = tokenizer.tokenize(sentence_kipi)
encoded_line = tokenizer.convert_tokens_to_ids(tokens)
decoded_line = tokenizer.convert_ids_to_tokens(encoded_line)
print("Length of vocab : ", len(tokenizer_vocab))
print("Input example : ", sentence_org)
print("Tokenized example : ", tokens)
print("Encoded line : ", encoded_line)
print("Decoded line : ", decoded_line)

Length of vocab :  35000
Input example :  본 고안은 주로 일회용 합성세제액을 집어넣어 밀봉하는 세제액포의 내부를 원호상으로 열중착하되 세제액이 배출되는 절단부 쪽으로 내벽을 협소하게 형성하여서 내부에 들어있는 세제액을 잘짜질 수 있도록 하는 합성세제 액포에 관한 것이다.
Tokenized example :  ['본', '고안', '은', '주로', '일회용', '합성', '##세제', '##액', '을', '집어넣', '어', '밀봉', '하', '는', '세제', '##액', '##포', '의', '내부', '를', '원호', '상', '으로', '열', '##중', '착하', '되', '세제', '##액', '이', '배출', '되', '는', '절단부', '쪽', '으로', '내벽', '을', '협소', '하', '게', '형성', '하', '여서', '내부', '에', '들', '어', '있', '는', '세제', '##액', '을', '잘', '짜', '질', '수', '있', '도록', '하', '는', '합성', '##세제', '액', '##포', '에', '관한', '것', '이', '다', '.']
Encoded line :  [1100, 5050, 1674, 6748, 10190, 5147, 27099, 2750, 1675, 28837, 1535, 5795, 2381, 519, 9646, 2750, 2940, 1687, 4771, 881, 9523, 1251, 4682, 1565, 2591, 20032, 618, 9646, 2750, 1692, 4977, 618, 519, 13247, 1886, 4682, 7485, 1675, 14947, 2381, 155, 4689, 2381, 7926, 4771, 1553, 657, 1535, 1703, 519, 9646, 2750, 1675, 1712, 1849, 1840, 1351, 1703, 4692, 2381, 519, 5147, 27099, 1512, 2940, 

# 4. 모델

In [ ]:
# from transformers import ElectraModel, ElectraTokenizer

# model = ElectraModel.from_pretrained("monologg/koelectra-base-discriminator")  # KoELECTRA-Base
# model = ElectraModel.from_pretrained("monologg/koelectra-small-discriminator")  # KoELECTRA-Small
# model = ElectraModel.from_pretrained("monologg/koelectra-base-v2-discriminator")  # KoELECTRA-Base-v2
# model = ElectraModel.from_pretrained("monologg/koelectra-small-v2-discriminator")  # KoELECTRA-Small-v2
# model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")  # KoELECTRA-Base-v3
# model = ElectraModel.from_pretrained("monologg/koelectra-small-v3-discriminator")  # KoELECTRA-Small-v3

In [14]:
from transformers import ElectraModel, ElectraTokenizer

# 모델 경로 (KIPI-KorPatELECTRA 경로 설정)
model_path = "/home/hee/project/graduate_paper/KIPI-KorPatELECTRA/KorPatELECTRA/PT"

# 토크나이저 및 모델 불러오기
tokenizer = ElectraTokenizer.from_pretrained(model_path)
model = ElectraModel.from_pretrained(model_path)

print("모델과 토크나이저가 성공적으로 불러와졌습니다.")

Some weights of the model checkpoint at /home/ui/project/graduate_paper/KIPI-KorPatELECTRA/KorPatELECTRA/PT were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


모델과 토크나이저가 성공적으로 불러와졌습니다.


# 5. 개체명 추출

In [ ]:
from functions import *
setup_logging()
get_env_var("IP_170")
logging.info("안뇽?")

TypeError: 'int' object is not callable